<h1>Ссылочные документы марки</h1>

In [22]:
import pandas as pd
import numpy as np
import pyodbc 

db_conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-JSR6TV0G;'
                      'Database=Pro_t_londonSQL;'
                      'Trusted_Connection=yes;')

In [23]:
df = pd.read_sql_query('SELECT * FROM Pro_t_londonSQL.dbo.Од_ссыл', db_conn)
df.shape

(293, 3)

In [24]:
df.head()

,код_марки,шифр_сд,прим
0,234,УЗ17,None
1,234,ЛП3,None
2,234,ЛП4,None
3,238,ЛП3,None
4,238,ЛП4,None


In [25]:
df = df.replace("", np.nan)

In [26]:
df.isna().sum()

код_марки      0
шифр_сд        0
прим         293
dtype: int64

Все примечания = null

Удаляем столбец на данный момент

In [27]:
df = df.drop(["прим"], axis=1)
df.head()

,код_марки,шифр_сд
0,234,УЗ17
1,234,ЛП3
2,234,ЛП4
3,238,ЛП3
4,238,ЛП4


In [28]:
df["шифр_сд"].value_counts()

ПБ14    50
УЗ17    37
ПБ3     16
ПБ4     16
ЛП4     16
ЛП3     15
УЗ20    13
ПБ15    13
ПБ4     12
ЛП4     12
КЛ11    12
ПК10    12
ЛП5      9
ЛП10     8
ЛП3      7
ЛП9      4
УЗ22     4
ЛГ       4
уз17     4
лп4      3
ЛП12     3
ЛП11     3
ЗС       2
ПБ3      2
ДО       2
СФ1      2
ПБ13     1
СК1      1
ПРЧ1     1
ВБ2      1
УЗ19     1
М        1
ОЛ35     1
1        1
тг11     1
СМ72     1
ПЛ1      1
пб4      1
Name: шифр_сд, dtype: int64

1? Аномалия?

In [29]:
df[df["шифр_сд"] == "1"]

,код_марки,шифр_сд
206,1870,1


In [30]:
df= df[df["шифр_сд"] != "1"]
df["шифр_сд"] = df["шифр_сд"].str.upper()
df["шифр_сд"].value_counts()

ПБ14    50
УЗ17    41
ЛП4     19
ПБ4     16
ПБ3     16
ЛП3     15
ПБ4     13
ПБ15    13
УЗ20    13
ЛП4     12
КЛ11    12
ПК10    12
ЛП5      9
ЛП10     8
ЛП3      7
ЛП9      4
ЛГ       4
УЗ22     4
ЛП12     3
ЛП11     3
ДО       2
СФ1      2
ПБ3      2
ЗС       2
ТГ11     1
ОЛ35     1
ВБ2      1
М        1
ПЛ1      1
СК1      1
УЗ19     1
СМ72     1
ПБ13     1
ПРЧ1     1
Name: шифр_сд, dtype: int64

In [31]:
m_ids = pd.read_csv("mark_ids.csv")
m_ids = m_ids.drop(["Unnamed: 0"], axis=1)
m_ids = m_ids.values.flatten().tolist()
df = df[df["код_марки"].isin(m_ids)]
df.shape

(0, 2)

Нет ссылочных документов для текущих марок

<h2>Random generation of linked documents</h2>

In [32]:
import random

In [33]:
for i in range(1, 301):
    df = df.append(pd.Series(), ignore_index=True)

C:\Users\sigla\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [34]:
df.loc[:, "код_марки"] = random.choices(m_ids, k=300)
df.loc[:, "шифр_сд"] = random.choices(range(1, 311), k=300)

In [35]:
df = df.drop_duplicates(subset=["код_марки", "шифр_сд"])
df.head()

,код_марки,шифр_сд
0,4299,8
1,5303,39
2,4413,245
3,5401,266
4,4364,214


<h2>Id</h2>

In [36]:
df.insert(0, 'id', range(1, len(df) + 1))

<h2>Меняем названия столбцов</h2>

In [37]:
df = df.rename(columns={"код_марки": "mark_id",
                        "шифр_сд": "linked_doc_id",
                        "прим": "note"
                       })
df

,id,mark_id,linked_doc_id
0,1,4299,8
1,2,5303,39
2,3,4413,245
3,4,5401,266
4,5,4364,214
...,...,...,...
295,296,4580,306
296,297,5091,61
297,298,5522,36
298,299,5011,236


<h1>Postgres</h1>

Создание таблицы

In [38]:
from psycopg2 import connect, sql, DatabaseError
import psycopg2.extras as extras

from psycopg2.extensions import register_adapter, AsIs
register_adapter(np.int32, AsIs)
register_adapter(np.int64, AsIs)

DocumentsKM

In [39]:
# Connect
try:
    conn = connect (
        dbname = "documentskm",
        user = "postgres",
        host = "localhost",
        password = "password"
    )
    cursor = conn.cursor()
except Exception as err:
    cursor = None
    print("Psycopg2 error:", err)
    
# Check if the connection was valid
if cursor != None:
    print("Connection successful")

Connection successful


In [40]:
cursor.execute(open("sql/14.sql", "r").read())
conn.commit()

<h2>Вставка данных</h2>

In [41]:
def execute_values(conn, df, table):
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

In [42]:
execute_values(conn, df, "mark_linked_docs")

execute_values() done
